In [7]:
# import Dependancies
import pandas as pd
import numpy as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func, desc
import datetime

In [8]:
# switch to seaborn deafult display 
sns.set()

In [9]:
# vacation start and end dates
trip_start = '2020-09-01'
trip_end = '2021-09-16'

In [10]:
# create engine
engine = create_engine("sqlite:///hawaii.sqlite")

In [11]:
# automaps and reflects database
Base = automap_base()
Base.prepare(engine, reflect=True)

In [12]:
# connects to database & initializes inspector
conn = engine.connect()
inspector = inspect(engine)

In [21]:
# creates class for each datframe
Measurement = Base.classes.measurement
Station = Base.classes.station

AttributeError: measurement

In [20]:
# inspects database
Base.classes.keys()

[]

In [17]:
session = Session(engine)

In [18]:
# data from previous years rainfall
prerain_an = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= "2018-09-01").\
    filter(Measurement.date <= "2019-09-16").all()

NameError: name 'Measurement' is not defined

In [ ]:
# create df
prerain_df =pd.DataFrame(prerain_an)

In [ ]:
# convert date to datatime object
prerain_df['date'] = pd.to_datatime(prerain_df['df'])

In [ ]:
# reindexes dataframe and rename column
prerain_df = prerain_df.set_index('date')
prerain_df.rename(columns = {'prcp': 'precipitation'}, inplace=True)
prerain_df.head()

In [ ]:
# plot
prerain_df.plot(figsize=(15, 7),
               sort_column=True,
               rot=45,
               use_index=True,
               title="Precipition in Hawaii from 09/01/2018 to 09/16/2019"
               legend=True
               fontsize=16,
               grid=True,
               )
plt.xlabel('Date', fontsize=20)
plt.ylabel('Amount', fontsize=20)
plt.show

In [ ]:
# stats
prerain_df.describe()

In [ ]:
# Station Analysis
station_count = session.query(Station.id).count() 
print(f'There are {station_count} weather stations in Hawaii.')

In [ ]:
# finds station activity and sorts from most active to least active.
station_activity = session.query(Measurement.station, Station.name, func.count(Measurement.tobs)).\
filter(Measurement.station == Station.station).group_by(Measurement.station).order_by(func.count(Measurement.tobs).desc()).all()

In [ ]:
# display station activity
station_activity

In [ ]:
# extract the stations with the most activity
for row in reversed(station_activity):  
    most_active_station=row.station
    most_active_name=str(row.name)

In [ ]:
# finds tobs for most active station
most_active_station_tobs = session.query(Measurement.tobs).\
filter(Measurement.station == most_active_station, Measurement.station == Station.station,\
       Measurement.date >="2016-08-24", Measurement.date <="2017-08-23").all()

In [ ]:
# extract temps from the query
temperatures = [temp[0] for temp in most_active_station_tobs] 

In [ ]:
# plot
plt.hist(temperatures, bins=12)
plt.title("Temperature Observation for %s from 2018-09-01 to 2019-09-16" % (most_active_name))
plt.ylabel('Frequency', fontsize=16)
labels = ['tobs']
plt.legend(labels)
plt.show()

In [ ]:
# Temperture Analysis
def calc_temps(start, end):
    temp_list = []
    query = session.query(Measurement.tobs).filter(Measurement.date>=start, Measurement.date<=end).all()
    temperatures = [temp[0] for temp in query]
    temp_list.append(np.mean(temperatures))
    temp_list.append(min(temperatures))
    temp_list.append(max(temperatures))
    return temp_list

In [ ]:
def get_prev_year_date(target):
    try:
        xl = target.split('-')
        xc = (str(int(xl[0])-1)) + '-' + xl[1] + '-' + xl[2]
        return xc
    except:
        return target

In [ ]:
# get pervious year data
start_conv = get_prev_year_date(trip_start_date)
end_conv = get_prev_year_date(trip_end_date)

In [ ]:
# call temp cals to plot data
temp_list = calc_temps(start_conv, end_conv) 

In [ ]:
# displays list of plot data
temp_list

In [ ]:
# plot
plt.figure(figsize=(3,5))               
plt.bar(1, temp_list[0], yerr=(temp_list[2] - temp_list[1]))
plt.ylabel("Temperature (F)", weight="bold")
plt.title("Avg Trip Temp: %s to %s" % (start_conv, end_conv))
plt.show()